In [162]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [163]:
db = pd.read_excel('dados_vendas.xlsx')

#Deleting nascimento
db.drop("NACIMENTO", axis=1, inplace=True)

#Deleting possible errors
db = db[(db.IDADE > 15) & (db.IDADE < 95)]
db = db[db.CANAL == 'VENDAWEB']



In [164]:
db = db.dropna()
count = 313924
        
cliente = db.iloc[1]
cliente.IDADE

35

In [165]:
data = db.copy()


In [166]:
data.describe()

,COD_FILIAL,DATA_COMPRA,IDADE,COD_PRODUTO,QUANTIDADE
count,291414.000000,2.914140e+05,291414.000000,291414.000000,291414.000000
mean,27.666227,2.019408e+07,42.607730,24890.416991,1.027408
std,17.848502,4.339313e+03,14.158595,1971.347532,0.415675
min,1.000000,2.019080e+07,16.000000,10216.000000,1.000000
25%,13.000000,2.019093e+07,32.000000,24913.000000,1.000000
50%,24.000000,2.019113e+07,41.000000,25726.000000,1.000000
75%,40.000000,2.020012e+07,52.000000,25998.000000,1.000000
max,65.000000,2.020052e+07,94.000000,26727.000000,58.000000


In [168]:
def del_gender_inconsistence(dataset):
    DB = dataset.groupby(['SEXO'])['COD_CLIENTE']   fem = dataset.COD_CLIENTE.isin(DB.get_group('F'))
    mas = dataset.COD_CLIENTE.isin(DB.get_group('M'))
    to_be_deleted = dataset[fem & mas].index
    db_cleaned = dataset.drop(to_be_deleted).reset_index()
    return db_cleaned

In [169]:
db_gender_cleaned = del_gender_inconsistence(db)
#db_gender_cleaned.to_csv('db_gender_cleaned.csv', sep=';')
db_gender_cleaned.describe()

,index,COD_FILIAL,DATA_COMPRA,IDADE,COD_PRODUTO,QUANTIDADE
count,291414.000000,291414.000000,2.914140e+05,291414.000000,291414.000000,291414.000000
mean,170574.337177,27.666227,2.019408e+07,42.607730,24890.416991,1.027408
std,87969.398919,17.848502,4.339313e+03,14.158595,1971.347532,0.415675
min,1.000000,1.000000,2.019080e+07,16.000000,10216.000000,1.000000
25%,95560.250000,13.000000,2.019093e+07,32.000000,24913.000000,1.000000
50%,170926.500000,24.000000,2.019113e+07,41.000000,25726.000000,1.000000
75%,246731.750000,40.000000,2.020012e+07,52.000000,25998.000000,1.000000
max,321740.000000,65.000000,2.020052e+07,94.000000,26727.000000,58.000000


In [170]:
clustered = db_gender_cleaned.groupby(['IDADE'])['COD_CLIENTE']
clustered.describe()

,count,unique,top,freq
IDADE,,,,
16,70,60,99CZQX01,3
17,335,232,9NYCYI01,51
18,994,856,YYYBKD01,5
19,2278,1883,9ORB5C01,7
20,3433,2795,9OAYT101,12
...,...,...,...,...
90,39,33,81315101,3
91,13,12,YYYCPP01,2
92,13,13,9OQDY301,1


In [171]:
groups = clustered.groups

In [175]:
groups_list = []
for group in groups:
    groups_list.append(group)    
    
print(groups_list)

[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94]


In [188]:
voudeletar = []
for age in tqdm(groups_list):
    idade = db_gender_cleaned.COD_CLIENTE.isin(DB.get_group(age))
    for age2 in groups_list:
        if(age2 == 94):
            continue
        else:            
            idade_checada = db_gender_cleaned.COD_CLIENTE.isin(DB.get_group(age2 + 1))
            user_inconsistente = db_gender_cleaned[idade & idade_checada].index
            limpo = db_gender_cleaned.drop(user_inconsistente).reset_index()


In [189]:
limpo

,level_0,index,COD_FILIAL,ORCAMENTO,DATA_COMPRA,HORA,COD_CLIENTE,SEXO,IDADE,COD_PRODUTO,NOME_PRODUTO,CLASSIFICACAO,QUANTIDADE,CANAL
0,0,1,1,99AW74,20200515,2020-05-15T11:14:54,05190001,M,34,25172,SMARTPHONE LG K9 TV LM-X210BMW PRETO,CELULAR SMARTPHONE SMARTPHONE ...,1,VENDAWEB
1,1,5,39,355603,20200515,2020-05-15T14:09:47,37578201,F,35,25367,TABLET NB729 MINI MS40G BRANCO,TECNOLOGIA INFORMATICA TABLET ...,1,VENDAWEB
2,2,7,17,99BYU2,20200515,2020-05-15T11:01:29,93168801,M,30,25354,"TELEVISOR SMART 32"" UN32J4290 SAMSUNG",TECNOLOGIA TV E VIDEO SMART TV ...,1,VENDAWEB
3,3,9,38,890640,20200515,2020-05-15T14:23:09,62982901,M,20,26584,SMARTPHONE MOTOROLA MOTO G8 PLUS 64GB CEREJA,CELULAR SMARTPHONE SMARTPHONE ...,1,VENDAWEB
4,4,11,12,99OH16,20200515,2020-05-15T15:10:46,54509401,F,32,21647,DVD D-15 KARAOKE MONDIAL,TECNOLOGIA TV E VIDEO DVD ...,1,VENDAWEB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291400,291409,321736,16,418367,20190801,2019-08-01T10:41:03,9OI04701,M,47,24716,RECEPTOR MIDIA BOX HDTV B3,TECNOLOGIA TV E VIDEO RECEPTOR ...,1,VENDAWEB
291401,291410,321737,56,114991,20190801,2019-08-01T13:42:49,99FZPR01,M,32,25981,SMARTPHONE SAMSUNG GAL A10 A105 32GB PRETO,CELULAR SMARTPHONE SMARTPHONE ...,1,VENDAWEB
291402,291411,321738,13,99X8IJ,20190801,2019-08-01T09:33:00,9NYVEB01,M,35,25981,SMARTPHONE SAMSUNG GAL A10 A105 32GB PRETO,CELULAR SMARTPHONE SMARTPHONE ...,1,VENDAWEB
291403,291412,321739,9,982718,20190801,2019-08-01T12:01:18,99K0N601,M,41,25727,SMARTPHONE SAMSUNG GAL J2 CORE 16GB PRATA,CELULAR SMARTPHONE SMARTPHONE ...,1,VENDAWEB


In [196]:

limpo.to_csv('db_limpa.csv', sep=';')

KeyError: 'Unnamed'

In [198]:
limpo

,COD_CLIENTE,COD_PRODUTO,NOME_PRODUTO,QUANTIDADE,CANAL
0,05190001,25172,SMARTPHONE LG K9 TV LM-X210BMW PRETO,1,VENDAWEB
1,37578201,25367,TABLET NB729 MINI MS40G BRANCO,1,VENDAWEB
2,93168801,25354,"TELEVISOR SMART 32"" UN32J4290 SAMSUNG",1,VENDAWEB
3,62982901,26584,SMARTPHONE MOTOROLA MOTO G8 PLUS 64GB CEREJA,1,VENDAWEB
4,54509401,21647,DVD D-15 KARAOKE MONDIAL,1,VENDAWEB
...,...,...,...,...,...
291400,9OI04701,24716,RECEPTOR MIDIA BOX HDTV B3,1,VENDAWEB
291401,99FZPR01,25981,SMARTPHONE SAMSUNG GAL A10 A105 32GB PRETO,1,VENDAWEB
291402,9NYVEB01,25981,SMARTPHONE SAMSUNG GAL A10 A105 32GB PRETO,1,VENDAWEB
291403,99K0N601,25727,SMARTPHONE SAMSUNG GAL J2 CORE 16GB PRATA,1,VENDAWEB
